### 1. Importing Libraries:

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sklearn
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
import re
from sklearn.preprocessing import (
	FunctionTransformer
)

In [18]:
import nltk

from nltk.stem import PorterStemmer # for stemming

from nltk.stem import WordNetLemmatizer # for lemmatization

from nltk.corpus import stopwords

In [19]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [20]:
stop_words = set(stopwords.words('english'))

In [21]:
plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')

### 2. Display Setting:

In [22]:
pd.set_option("display.max_columns", None)

In [23]:
sklearn.set_config(transform_output="pandas")

### 3. Read The Data:

In [24]:
df=pd.read_csv(r'U:\nlp_project\gender_classification\data\gender_classifier.csv',encoding='latin1')

In [25]:
df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,description,fav_number,gender_gold,link_color,name,profile_yn_gold,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,i sing my own rhythm.,0,NaN,08C2C2,sheezy0,NaN,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,I'm the author of novels filled with family dr...,68,NaN,0084B4,DavdBurnett,NaN,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,louis whining and squealing and all,7696,NaN,ABB8C2,lwtprettylaugh,NaN,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",202,NaN,0084B4,douggarland,NaN,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,37318,NaN,3B94D9,WilfordGemma,NaN,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


In [26]:
df.columns.to_list()

['_unit_id',
 '_golden',
 '_unit_state',
 '_trusted_judgments',
 '_last_judgment_at',
 'gender',
 'gender:confidence',
 'profile_yn',
 'profile_yn:confidence',
 'created',
 'description',
 'fav_number',
 'gender_gold',
 'link_color',
 'name',
 'profile_yn_gold',
 'profileimage',
 'retweet_count',
 'sidebar_color',
 'text',
 'tweet_coord',
 'tweet_count',
 'tweet_created',
 'tweet_id',
 'tweet_location',
 'user_timezone']

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20050 entries, 0 to 20049
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _unit_id               20050 non-null  int64  
 1   _golden                20050 non-null  bool   
 2   _unit_state            20050 non-null  object 
 3   _trusted_judgments     20050 non-null  int64  
 4   _last_judgment_at      20000 non-null  object 
 5   gender                 19953 non-null  object 
 6   gender:confidence      20024 non-null  float64
 7   profile_yn             20050 non-null  object 
 8   profile_yn:confidence  20050 non-null  float64
 9   created                20050 non-null  object 
 10  description            16306 non-null  object 
 11  fav_number             20050 non-null  int64  
 12  gender_gold            50 non-null     object 
 13  link_color             20050 non-null  object 
 14  name                   20050 non-null  object 
 15  pr

In [28]:
df.isnull().sum()

_unit_id                     0
_golden                      0
_unit_state                  0
_trusted_judgments           0
_last_judgment_at           50
gender                      97
gender:confidence           26
profile_yn                   0
profile_yn:confidence        0
created                      0
description               3744
fav_number                   0
gender_gold              20000
link_color                   0
name                         0
profile_yn_gold          20000
profileimage                 0
retweet_count                0
sidebar_color                0
text                         0
tweet_coord              19891
tweet_count                  0
tweet_created                0
tweet_id                     0
tweet_location            7485
user_timezone             7798
dtype: int64

In [29]:
df.shape

(20050, 26)

In [30]:
df[df['tweet_count']==1].shape

(26, 26)

### 4. Data Cleaning:

In [31]:
df[['description','text']]

,description,text
0,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...
1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...
2,louis whining and squealing and all,i absolutely adore when louis starts the songs...
3,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...
4,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...
...,...,...
20045,(rp),"@lookupondeath ...Fine, and I'll drink tea too..."
20046,"Whatever you like, it's not a problem at all. ...",Greg Hardy you a good player and all but don't...
20047,#TeamBarcelona ..You look lost so you should f...,You can miss people and still never want to se...
20048,Anti-statist; I homeschool my kids. Aspiring t...,@bitemyapp i had noticed your tendency to pee ...


#### 4.1 description and text:

In [32]:
df.description

0                                    i sing my own rhythm.
1        I'm the author of novels filled with family dr...
2                      louis whining and squealing and all
3        Mobile guy.  49ers, Shazam, Google, Kleiner Pe...
4        Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...
                               ...                        
20045                                                 (rp)
20046    Whatever you like, it's not a problem at all. ...
20047    #TeamBarcelona ..You look lost so you should f...
20048    Anti-statist; I homeschool my kids. Aspiring t...
20049                       Teamwork makes the dream work.
Name: description, Length: 20050, dtype: object

In [33]:
df.text

0        Robbie E Responds To Critics After Win Against...
1        ÛÏIt felt like they were my friends and I was...
2        i absolutely adore when louis starts the songs...
3        Hi @JordanSpieth - Looking at the url - do you...
4        Watching Neighbours on Sky+ catching up with t...
                               ...                        
20045    @lookupondeath ...Fine, and I'll drink tea too...
20046    Greg Hardy you a good player and all but don't...
20047    You can miss people and still never want to se...
20048    @bitemyapp i had noticed your tendency to pee ...
20049    I think for my APUSH creative project I'm goin...
Name: text, Length: 20050, dtype: object

In [34]:
lemmatizer = WordNetLemmatizer()

def cleaning(X):
    url_regex = r"https?://(www\.)?(\w+)(\.\w+)+(\/[\w\-.,]*)*"
    special_character_regex = r'[()@+#.;:"\'!?&_\-]'

    return (X
            # Apply transformations to each column
            .pipe(lambda x: x.assign(**{
                col: x[col]
                    .apply(lambda y: re.sub('[^a-zA-Z]', ' ', str(y)))  # Remove non-alphabetic chars
                    .replace(url_regex, " ", regex=True)  # Remove URLs
                    .replace(special_character_regex, " ", regex=True)  # Remove special characters
                    .str.lower()  # Convert to lowercase
                for col in X.columns.to_list()
            }))
            # Apply lemmatization and stopword removal
            .pipe(lambda x: x.assign(**{
                col: x[col].apply(lambda y: ' '.join([
                    lemmatizer.lemmatize(word) for word in y.split() if word.lower() not in stop_words
                ]))
                for col in X.columns.to_list()
            }))
            .drop(['_unit_id','_last_judgment_at','created','fav_number','profileimage','retweet_count','tweet_coord',
                   '_trusted_judgments', 'tweet_count', 'tweet_created', 'tweet_id', 'tweet_location', 'user_timezone', 
                   '_golden','_unit_state', 'gender_gold', 'link_color', 'name', 'profile_yn_gold', 'sidebar_color', 
                   'profile_yn', 'profile_yn:confidence','gender:confidence'], axis=1)
            .dropna()
    )


In [35]:
FunctionTransformer(func=cleaning).fit_transform(df)

,gender,description,text
0,male,sing rhythm,robbie e responds critic win eddie edward worl...
1,male,author novel filled family drama romance,felt like friend living story http co arnge yh...
2,male,louis whining squealing,absolutely adore louis start song hit hard fee...
3,male,mobile guy er shazam google kleiner perkins ya...,hi jordanspieth looking url use ifttt typicall...
4,female,ricky wilson best frontman kaiser chief best b...,watching neighbour sky catching neighbs xxx xxx
...,...,...,...
20045,female,rp,lookupondeath fine drink tea love
20046,male,whatever like problem chargernation foreverroy...,greg hardy good player get face dez bryant lea...
20047,male,teambarcelona look lost follow follow heart br...,miss people still never want see
20048,female,anti statist homeschool kid aspiring thoughtle...,bitemyapp noticed tendency pee carpet want say...


In [36]:
cleaning_transformer = Pipeline(steps=[
    ("part1", FunctionTransformer(func=cleaning))
])

In [37]:
cleaned_data = cleaning_transformer.fit_transform(df)

In [38]:
# nltk.download()

In [46]:
cleaned_data.shape

(20050, 3)

In [47]:
cleaned_data.to_csv(r'U:\nlp_project\gender_classification\data\cleaned_data.csv')

In [48]:
pd.read_csv(r'U:\nlp_project\gender_classification\data\cleaned_data.csv')

,Unnamed: 0,gender,description,text
0,0,male,sing rhythm,robbie e responds critic win eddie edward worl...
1,1,male,author novel filled family drama romance,felt like friend living story http co arnge yh...
2,2,male,louis whining squealing,absolutely adore louis start song hit hard fee...
3,3,male,mobile guy er shazam google kleiner perkins ya...,hi jordanspieth looking url use ifttt typicall...
4,4,female,ricky wilson best frontman kaiser chief best b...,watching neighbour sky catching neighbs xxx xxx
...,...,...,...,...
20045,20045,female,rp,lookupondeath fine drink tea love
20046,20046,male,whatever like problem chargernation foreverroy...,greg hardy good player get face dez bryant lea...
20047,20047,male,teambarcelona look lost follow follow heart br...,miss people still never want see
20048,20048,female,anti statist homeschool kid aspiring thoughtle...,bitemyapp noticed tendency pee carpet want say...


In [42]:
# input_variables = cleaned_data[['description', 'text']]

In [43]:
# input_variables.head()

In [44]:
# target_variables = cleaned_data['gender']

In [45]:
# target_variables